In [1]:
# Otras librerías utilizadas en este ejemplo.

from pathlib import Path

import pandas as pd
import spacy

In [2]:
# Principales objetos de banrep necesarios para ejemplo tópicos.

from banrep.corpus import MiCorpus
from banrep.diagnosticos import docs_topicos, topico_dominante, palabras_probables
from banrep.diagnosticos import corpus_stats, corpus_tokens
from banrep.io import Textos, leer_palabras
from banrep.topicos import crear_ldas, calcular_coherencias
from banrep.utils import crear_directorio

In [3]:
# Inicializar un modelo spacy, en este caso español mediano.

nlp = spacy.load('es_core_news_md')

In [4]:
# Uso de función: leer_palabras
# Definir variables a usar como filtros en la creación de corpus.
# Stopwords ubicadas en hoja "banrep_es" de archivo excel.
# Columna "type" determina el grupo y "word" las palabras.

pathstops = '/Users/tombito/Dropbox/datasets/wordlists/stopwords.xlsx'
palabras = leer_palabras(pathstops, 'banrep_es', col_grupo="type", col_palabras="word")
stops = palabras.get("stopword")

tags = ['NUM', 'PUNCT', 'SYM']

filtros = dict(stopwords=stops, postags=tags, entities=None, is_alpha=True)

In [5]:
# Uso de función: leer_palabras
# Definir listas de palabras (wordlists) cuya presencia se quiere evaluar en los textos.
# En este ejemplo, listas de palabras que representan "mejora" y "deterioro".
# Palabras ubicadas en columna "word" de hoja BANREP de archivo excel, agrupadas por columna "type".

pathwl = '/Users/tombito/Dropbox/datasets/wordlists/sentimiento.xlsx'
wordlists = leer_palabras(pathwl, 'BANREP', col_grupo="type", col_palabras="word")


# Cuantas palabras en cada grupo...

for tipo in wordlists:
    print(f'{len(wordlists.get(tipo))} palabras en grupo {tipo}')

1899 palabras en grupo deterioro
359 palabras en grupo mejora


In [6]:
# Uso de clase: Textos
# Definir los textos a procesar como documentos del corpus (archivos txt).
# chars=69 implica que líneas de longitud inferior a 70 caracteres serán excluidas.
# parrafos=True implica considerar cada párrafo de cada archivo txt como un documento.

dir_datos = Path('../datasets/inflacion/')

datos = Textos(dir_datos, aleatorio=False, chars=69, parrafos=True)
print(datos)

Textos de directorio inflacion, 4548 textos leídos.


In [7]:
# Uso de clase: MiCorpus
# Inicializa corpus con la información requerida.
# long=9 implica ignorar frases que tienen menos de 10 tokens.

corpus = MiCorpus(nlp, datos=datos, filtros=filtros, long=9, wordlists=wordlists)

print(corpus)

Corpus con 4548 docs y 3234 palabras únicas.


In [8]:
# Uso de función: crear_ldas
# Ejemplo crea modelos de tópicos para 5, 10, y 15 tópicos.
# Diferentes parámetros para modelos LDA pueden consultarse en documentación Gensim.
# https://radimrehurek.com/gensim/models/ldamodel.html - class gensim.models.ldamodel.LdaModel

n_topicos = (5, 10, 15)
params = dict(passes=5, alpha='auto', eta='auto', random_state=100)

modelos = [lda for lda in crear_ldas(corpus, n_topicos, params)]

In [9]:
# Uso de función: calcular_coherencias
# Calcular Coherence Score de cada modelo para determinar el "mejor".
# Mayor Coherence Score es mejor modelo.

scores = [score for score in calcular_coherencias(modelos, corpus)]

mejor_score = max(scores)
cual = scores.index(mejor_score)
mejor_n = n_topicos[cual]

print(f"Modelo de {mejor_n} tópicos mejor Coherence Score: {mejor_score}")

Modelo de 5 tópicos mejor Coherence Score: 0.4730777032986858


In [10]:
# Uso de función: crear_directorio
# Guardar lo útil: modelos bigramas y trigramas, diccionario.

# Crear directorio de salida
dir_salida = crear_directorio('topicos')

#Guardar modelos de bigramas y trigramas
corpus.ngrams.get('bigrams').save(str(dir_salida.joinpath('bigrams')))
corpus.ngrams.get('trigrams').save(str(dir_salida.joinpath('trigrams')))

# Guardar diccionario
corpus.id2word.save(str(dir_salida.joinpath('id2word')))

In [11]:
# Guardar mejor modelo de tópicos y su visualización

dirtopic = dir_salida.joinpath(f'{mejor_n:0>2}')
crear_directorio(dirtopic)

modelo = modelos[cual]
modelo.save(str(dirtopic.joinpath('topicos.lda')))

# En análisis de tópicos se suele usar PyLDAvis para visualizar resultados...
# Hay que importar la librería para eso.

import warnings

import pyLDAvis
import pyLDAvis.gensim

# Gráfica LDAvis de tópicos y sus palabras

bow = list(corpus)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    vis = pyLDAvis.gensim.prepare(modelo, bow, corpus.id2word, sort_topics=False)

pyLDAvis.save_html(vis, str(dirtopic.joinpath('topicos.html')))

In [12]:
# Uso de función: docs_topicos
# Usemos el mejor modelo para ver...
# Distribución de probabilidad de tópicos en cada documento...

doctopics = docs_topicos(modelo, corpus)

doctopics.tail()

,0,1,2,3,4
d0025p226,0.013295,0.948745,0.016371,0.011316,0.010273
d0025p227,NaN,NaN,0.926443,0.064076,NaN
d0025p228,NaN,NaN,0.987917,NaN,NaN
d0025p229,NaN,NaN,0.984735,NaN,NaN
d0025p230,NaN,NaN,NaN,0.398867,0.587020


In [13]:
# Uso de función: topico_dominante
# Cada tópico es dominante en cuantos documentos?
# Tópico 7 es el de mayor probabilidad en el 15% de los documentos...

dominante = topico_dominante(doctopics)

dominante.head()

,topico,docs
0,2,0.2511
1,0,0.2322
2,1,0.2234
3,4,0.1882
4,3,0.1051


In [14]:
# Uso de función: palabras_probables
# Cuales son las palabras más probables en cada uno de los tópicos dominantes...

dfs = []
for topico in dominante['topico']:
    df = palabras_probables(modelo, topico, n=20)
    dfs.append(df)

palabras = pd.concat(dfs, ignore_index=True)

palabras.head()

,palabra,probabilidad,topico
0,inflación,0.033681,2
1,pronósticos,0.012194,2
2,pronóstico,0.010738,2
3,tasas_interés,0.009935,2
4,expectativas,0.009268,2


In [15]:
# Estadísticas del corpus
stats = corpus_stats(corpus)
stats.tail()

,doc_id,archivo,fuente,frases,palabras
4543,d0025p226,2004-12.txt,inflacion,1,10
4544,d0025p227,2004-12.txt,inflacion,2,40
4545,d0025p228,2004-12.txt,inflacion,2,39
4546,d0025p229,2004-12.txt,inflacion,3,32
4547,d0025p230,2004-12.txt,inflacion,3,31


In [16]:
# Estadísticas de tokens
tokens = corpus_tokens(corpus)
tokens.tail()

,doc_id,sent_id,tok_id,word,pos,deterioro,mejora
143729,d0025p230,3,87,transables,NOUN,False,False
143730,d0025p230,3,91,reforzaría,VERB,False,False
143731,d0025p230,3,93,sesgo,NOUN,False,False
143732,d0025p230,3,98,política,NOUN,False,False
143733,d0025p230,3,99,monetaria,ADJ,False,False
